In [55]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
conf=SparkConf().setAppName("miniProject").setMaster("local[*]")
sc=SparkContext.getOrCreate(conf)

In [56]:
# 测试 import 等配置是否成功
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [96]:
# 读取数据
lines = sc.textFile('res/million_user_log.csv')

In [110]:
# 统计各省销售最好的产品类别前十（销售最多前10的产品类别）

# 购买行为
buy_action = lines.filter(lambda x: x.split(',')[7]=="2")
# 转换为（‘省份，类别’，次数）
groupby_provincebrand = buy_action.map(lambda x: (x.split(',')[10]+','+x.split(',')[2], 1)).reduceByKey(lambda a, b : a + b)
# 转换为（省份，（类别，次数））
groupby_province = groupby_provincebrand.map(lambda x: (x[0].split(',')[0],(x[0].split(',')[1],x[1])))
# 将 item 按省份集合起来
each_province = groupby_province.groupByKey()
# 在个省份内按销量排序，取出最高十条
result = each_province.map(lambda x: (x[0], sorted(list(x[1]), key=lambda y: y[1], reverse=True)[:10]))

result.collect()

[('天津市',
  [('1208', 130),
   ('656', 115),
   ('1213', 89),
   ('602', 88),
   ('662', 82),
   ('1142', 80),
   ('389', 75),
   ('737', 74),
   ('177', 71),
   ('664', 67)]),
 ('福建',
  [('1208', 120),
   ('656', 116),
   ('662', 83),
   ('177', 80),
   ('602', 76),
   ('1213', 72),
   ('389', 72),
   ('1142', 71),
   ('1401', 67),
   ('737', 64)]),
 ('山西',
  [('656', 123),
   ('1208', 119),
   ('1401', 83),
   ('602', 79),
   ('177', 78),
   ('1213', 77),
   ('664', 73),
   ('1142', 73),
   ('420', 72),
   ('737', 70)]),
 ('安徽',
  [('656', 141),
   ('1208', 120),
   ('602', 101),
   ('737', 85),
   ('1213', 84),
   ('1401', 80),
   ('662', 77),
   ('664', 68),
   ('420', 67),
   ('1142', 66)]),
 ('上海市',
  [('656', 125),
   ('602', 108),
   ('1208', 104),
   ('1142', 83),
   ('177', 81),
   ('1213', 76),
   ('1401', 75),
   ('662', 75),
   ('737', 73),
   ('389', 72)]),
 ('贵州',
  [('1208', 126),
   ('656', 110),
   ('602', 85),
   ('1213', 79),
   ('737', 77),
   ('1142', 77),
   ('389

In [112]:
# 统计各省的双十一前十热门销售产品（购买最多前10的产品）

# 购买行为
buy_action = lines.filter(lambda x: x.split(',')[7]=="2")
# 转换为（‘省份，类别’，次数）
groupby_provincebrand = buy_action.map(lambda x: (x.split(',')[10]+','+x.split(',')[1], 1)).reduceByKey(lambda a, b : a + b)
# 转换为（省份，（类别，次数））
groupby_province = groupby_provincebrand.map(lambda x: (x[0].split(',')[0],(x[0].split(',')[1],x[1])))
# 将 item 按省份集合起来
each_province = groupby_province.groupByKey()
# 在个省份内按销量排序，取出最高十条
result = each_province.map(lambda x: (x[0], sorted(list(x[1]), key=lambda y: y[1], reverse=True)[:10]))

result.collect()

[('天津市',
  [('1059899', 8),
   ('783997', 7),
   ('1100222', 6),
   ('317073', 6),
   ('191499', 6),
   ('698879', 5),
   ('823766', 5),
   ('493761', 5),
   ('864005', 5),
   ('3001', 5)]),
 ('山西',
  [('191499', 9),
   ('221663', 7),
   ('559647', 7),
   ('353560', 7),
   ('713695', 7),
   ('936203', 6),
   ('1059899', 6),
   ('349999', 5),
   ('654894', 5),
   ('15173', 5)]),
 ('云南',
  [('191499', 10),
   ('1059899', 7),
   ('48664', 5),
   ('349999', 5),
   ('655904', 5),
   ('1010145', 5),
   ('147751', 4),
   ('181387', 4),
   ('276750', 4),
   ('179830', 4)]),
 ('陕西',
  [('191499', 9),
   ('353560', 9),
   ('514725', 8),
   ('936203', 7),
   ('107407', 7),
   ('221663', 7),
   ('181387', 6),
   ('28895', 6),
   ('1091980', 6),
   ('1059899', 5)]),
 ('安徽',
  [('353560', 6),
   ('676215', 6),
   ('823766', 6),
   ('186456', 6),
   ('514725', 5),
   ('554408', 5),
   ('636863', 5),
   ('100215', 5),
   ('1059899', 5),
   ('801860', 5)]),
 ('青海',
  [('353560', 8),
   ('944554', 7),
 

In [23]:
# 读取数据为 dataframe 格式
df = spark.read.csv('res/million_user_log.csv', encoding='utf-8', header=True, inferSchema=True)
df.show()

+--------+-------+--------+---------+-----+-----+---+------+---+---+--------+
|buyer_id|good_id|category|seller_id|brand|month|day|action|age|sex|province|
+--------+-------+--------+---------+-----+-----+---+------+---+---+--------+
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  0|  1|    四川|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  7|  1|  重庆市|
|  328862| 807126|    1181|     1963| 6109|   11| 11|     0|  1|  0|  上海市|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     2|  6|  0|    台湾|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  6|  2|    甘肃|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  4|  1|    甘肃|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  5|  0|    浙江|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  3|  2|    澳门|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  7|  1|    台湾|
|  234512| 399860|     962|      305| 6300|   11| 11|     0|  4|  1|    安徽|
|  23451

In [27]:
# 创建视图，并执行简单的 sql 指令
df.createOrReplaceTempView("millions")
spark.sql('''select * from millions''').show()

+--------+-------+--------+---------+-----+-----+---+------+---+---+--------+
|buyer_id|good_id|category|seller_id|brand|month|day|action|age|sex|province|
+--------+-------+--------+---------+-----+-----+---+------+---+---+--------+
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  0|  1|    四川|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  7|  1|  重庆市|
|  328862| 807126|    1181|     1963| 6109|   11| 11|     0|  1|  0|  上海市|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     2|  6|  0|    台湾|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  6|  2|    甘肃|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  4|  1|    甘肃|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  5|  0|    浙江|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  3|  2|    澳门|
|  328862| 406349|    1280|     2700| 5476|   11| 11|     0|  7|  1|    台湾|
|  234512| 399860|     962|      305| 6300|   11| 11|     0|  4|  1|    安徽|
|  23451

In [113]:
# 查询双11那天浏览次数前十的品牌
spark.sql('''SELECT first(brand), count(*)as times FROM millions where action=0 group by brand order by times desc limit 10''').show()

+-------------------+-----+
|first(brand, false)|times|
+-------------------+-----+
|               1360|49151|
|               3738|10130|
|                 82| 9719|
|               1446| 9426|
|               6215| 8568|
|               1214| 8470|
|               5376| 8282|
|               2276| 7990|
|               1662| 7808|
|               8235| 7661|
+-------------------+-----+

